In [ ]:
import pandas as pd 
import numpy as np
from tqdm import tqdm  # Import the tqdm library

#Reads here are spliced + barcoded + unique molecules so there are a LOT fewer reads in that than in the BAM
allinfo = pd.read_csv('/gpfs/commons/groups/knowles_lab/Karin/data/isoform_cortex_longread_SC/Anoushka_allInfo/FCtx_Rep2_ONT_AllInfo.gz', sep='\t', header=None)

In [ ]:
allinfo.head()

In [ ]:
allinfo[2].unique()

In [ ]:
# get number of unique cells and how many of each type they are (column 2)
cells_types = allinfo[[0,2]].drop_duplicates()
cells_types.head()

In [ ]:
cells_types[2].value_counts()

In [ ]:
# load the gtf file 
gtf = pd.read_csv('/gpfs/commons/home/kisaev/leafcutter-sc/PCDH_gene_clusters.gtf', delimiter=r"\s+")
gtf.head()

In [ ]:
allinfo_exons = allinfo[[0,1,2, 3, 4, 6, 8, 10]]
allinfo_exons.head()

In [ ]:
# check via gene_id column if any of the pcdh genes are there using grep 

In [ ]:
# Split the exon coordinates by the delimiter ';%;'
allinfo_exons[8] = allinfo_exons[8].str.split(';%;')

In [ ]:
# Remove the first element from each exon coordinate list
allinfo_exons[8] = allinfo_exons[8].apply(lambda lst: lst[1:])
allinfo_exons.head()

In [ ]:
# Explode the list of exons into separate rows
new_df = allinfo_exons.explode(8)

In [ ]:
# split column 8 into 4 columns: chr, start, end and strand 
new_df[['chromosome', 'start', 'end', 'strand']] = new_df[8].str.split('_', expand=True)
new_df.head()


In [ ]:
# reorder columns so chromosome, start, end, strand, gene_id, exon_id are first
new_df_save = new_df[["chromosome", "start", "end", "strand"]].drop_duplicates()

# save gtf file as bed file to view in IGV
new_df_save.to_csv('/gpfs/commons/home/kisaev/leafcutter-sc/lonread_isoseq_rep2.bed', sep='\t', index=False, header=False)

In [ ]:
new_df.head()

In [ ]:
#chr5:140,821,540-140,822,267
new_df[(new_df['chromosome'] == 'chr5') & (new_df['strand'] == "+")]

In [ ]:
# now need to map to exons in gtf 
gtf.head()

In [ ]:
gtf["exon_id"] = gtf.chromosome + "_" + gtf.start.astype(str) + "_" + gtf.end.astype(str) + "_" + gtf.strand

In [ ]:
# reorder columns so chromosome, start, end, strand, gene_id, exon_id are first
gtf = gtf[["chromosome", "start", "end", "strand", "source"]]

# save gtf file as bed file to view in IGV
gtf.to_csv('/gpfs/commons/home/kisaev/leafcutter-sc/PCDH_gene_clusters.bed', sep='\t', index=False, header=False)

In [ ]:
gtf